In [1]:
# 피쳐 계산? 예를 들어 트레이닝 기간 내 [평균가격/최근 n개 가격]이 529 라고 하면 언더 100 오버 100 정도 잡고 패딩의 민/맥스가 각 패딩의 50% 정도 되는 지점에 (예를 들어 +50) 다시 업데이트
# source / target / target_in 
# 데이터 가로로 쭉 줄세우고 10개씩 index 0 부터 +1 반복해서 가져온 뒤에, 앞에서 5개 끊어서 소스/ 뒤에서 5개 끊어서 타겟으로 만듬
# 어떻게 피쳐 업데이트할건지에 대한 고민 (o)

In [ ]:
import numpy as np

In [ ]:
#데이터셋 생성
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        target = list(np.arange(source[0] + 5,source[0]+n_out+5))
#         print(source)
#         print(target)
        
        target_in = [0] + target[:-1]
#         print('target:',target)
#         print('target-1:',target[:-1])
#         print('target_in:',target_in)
        
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return np.array(X1), np.array(X2), np.array(y)

#원핫 디코딩
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

#모델 생성
def define_models(n_input, n_output, n_units):
    
    #인코더 정의
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    
    #디코더 정의
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model

# 시퀀스 프레딕트
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return np.array(output)